In [1]:
import pandas as pd
import requests as rq
import numpy as np
import pyarrow
from bs4 import BeautifulSoup as bs
import os
from dotenv import load_dotenv
import time
from datetime import datetime
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
import src.escrapeo as es

In [2]:
load_dotenv()

True

In [3]:
RUTA_SERVICE = os.getenv("RUTA_SERVICE")
URL_ESCRAPEO_INICIAL = os.getenv("URL_ESCRAPEO_INICIAL")

In [4]:
url_destinos= URL_ESCRAPEO_INICIAL #url destinos totales
sopa_destinos = es.crear_sopa(url_destinos) #creamos la sopa
#Extraemos todos los contenidos de la clase landing-destination-title
clase_landing_destination_title=sopa_destinos.find_all("div", {"class":"landing-destination-title"})
print(f'Existen {len(clase_landing_destination_title)} destinos diferentes')

Existen 119 destinos diferentes


In [5]:
destinos_totales = es.destinos_totales(clase_landing_destination_title) #escrapeo los destinos totales

In [6]:
df_destinos_totales = pd.DataFrame(destinos_totales)
df_destinos_totales.head()

,nombre_pais,num_viajes_ofertados,fecha_escrapeo
0,Albania,1,2025-04-08
1,Alemania,11,2025-04-08
2,Antigua,2,2025-04-08
3,Argentina,23,2025-04-08
4,Aruba,3,2025-04-08


In [7]:
#añado los destinos actuales a los destinos anteriores y guardo el DF con los destinos totales acumulados
ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS = os.getenv("ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS")
if os.path.exists(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS): #comprueba si existe un fichero ya con destinos
    df_existente_destinos = pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS) #en caso de existir lo importa
    df_combinado_destinos = pd.concat([df_existente_destinos,df_destinos_totales], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_destinos=df_destinos_totales #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_destinos.drop_duplicates(inplace=True)
df_combinado_destinos.to_pickle(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS) #guarda el fichero en formato formato pickle

In [8]:
viajes_totales_destinos = es.viajes_totales_destinos(destinos_totales) #escrapeo los viajes totales de todos  los destinos 

Albania
Alemania
Antigua
Argentina
Aruba
Australia
Austria
Bahamas
Barbados
Bélgica
Belice
Bolivia
Botsuana
Brasil
Bulgaria
Bután
Cabo Verde
Camboya
Canadá
Chile
China
Chipre
Colombia
Corea Del Sur
Costa Rica
Croacia
Curaçao
Dinamarca
Dominica
Ecuador
Egipto
El Salvador
Emiratos Árabes Unidos
Eslovaquia
Eslovenia
España
Estados Unidos
Estonia
Fiji
Filipinas
Finlandia
Francia
Georgia
Granada
Grecia
Guadalupe
Guatemala
Honduras
Hungría
India
Indonesia
Irlanda
Islandia
Islas Caimán
Islas Cook
Islas Feroe
Italia
Jamaica
Japón
Jordania
Kenia
Laos
Letonia
Lituania
Madagascar
Malasia
Maldivas
Malta
Marruecos
Martinica
Mauricio
México
Mongolia
Montenegro
Namibia
Nepal
Nicaragua
Noruega
Nueva Caledonia
Nueva Zelanda
Omán
Países Bajos
Panamá
Papúa Nueva Guinea
Perú
Polinesia Francesa
Polonia
Portugal
Puerto Rico
Qatar
Reino Unido
República Checa
República Dominicana
Reunión
Riviera Maya
Ruanda
Rumanía
Saint Martin
Samoa
Santa Lucía
Serbia
Seychelles
Singapur
Sri Lanka
Sudáfrica
Suecia
Suiza
Tail

In [9]:
df_viajes_totales_destinos= pd.DataFrame(viajes_totales_destinos)
df_viajes_totales_destinos.head()

,pais,nombre_viaje,duracion_viaje,itinerario,precio,fecha_escrapeo
0,Albania,Albania y Montenegro,8 Días / 7 Noches,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",1755,2025-04-08
1,Alemania,Berlín,4 Días / 3 Noches,Berlín,708,2025-04-08
2,Alemania,"Fly & Drive Austria, Entre Lagos y Montañas",8 Días / 7 Noches,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...",827,2025-04-08
3,Alemania,Fly & Drive Carlomagno a su aire,10 Días / 9 Noches,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...",857,2025-04-08
4,Alemania,Fly & Drive Tirol al Completo,8 Días / 7 Noches,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,...",894,2025-04-08


In [10]:
#añado los viajes actuales a los viajes anteriores y guardo el DF con los viajes totales de todos los destinos acumulados
ARCHIVO_GUARDAR_ESCRAPEO_VIAJES = os.getenv("ARCHIVO_GUARDAR_ESCRAPEO_VIAJES")
if os.path.exists(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES): #comprueba si existe un fichero ya con viajes
    df_existente_viajes_totales= pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES)#en caso de existir lo importa
    df_combinado_viajes_totales = pd.concat([df_existente_viajes_totales,df_viajes_totales_destinos], ignore_index=True) #combina el fichero anterior con el nuevo obtenido en el escrapeo
else:
    df_combinado_viajes_totales=df_viajes_totales_destinos  #si no existe fichero previo, entonces, crea uno con el obtenido en el escrapeo
df_combinado_viajes_totales.drop_duplicates(inplace=True)
df_combinado_viajes_totales.to_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES) #guarda el fichero en formato pickle

In [13]:
df_combinado_viajes_totales.shape

(2056, 6)